## By : Asmaa Alrefae

# Data Overview :



__The used data is Arabic news data , which is scraped from a very popular Sauid news called Sbaq.__

__It is contains 4136 news , With the following five features :__

- Title : The news title .
- Author Name : The name of the author .
- City : The graphical location of the news.
- Shares : How many times specific news where shared in the social media.
- News text : The body of the news.
- Label : The classification of the news ( locally / internationally / ...etc)

## Problem Statement :

__Create a model to predict the classification of a certian news , whether it is ex: locally ,Internationally ,..etc__

# Step1 :  Clean and Prepare data 

In [166]:
import requests
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer

In [167]:
# This has been re-scrapped in another Jupyter book as the given data were not accurate and complete 
# Not Accurate : The title , were not matching the news text 
# Not complete : The label were not mentioned
news = pd.read_csv('full_data.csv') 

In [168]:
# Delete ccoulmn name "Unnamed: 0" as it is duplicate 
del news['Unnamed: 0']

In [169]:
news.head(5)

,title,author_name,city,shares,news_text,label
0,الدفاع المدني بالرياض يناقش مع الهلال الأحمر...,صحيفة سبق الإلكترونية,الرياض,2,استقبل مدير الدفاع المدني بمنطقة الرياض، اللوا...,"['المملكة', 'محليات']"
1,"""أبوساق"": خطاب الملك بـ""الشورى"" وثيقة استرات...",وكالة الأنباء السعودية (واس),الرياض,4,عدَّ وزير الدولة عضو مجلس الوزراء لشؤون مجلس ا...,"['المملكة', 'محليات']"
2,المملكة ترفض تصريحات الحكومة الأمريكية بشأن ...,وكالة الأنباء السعودية (واس),الرياض,5,عبر مصدر مسؤول بوزارة الخارجية عن رفضه التام ل...,"['المملكة', 'محليات']"
3,ولي العهد يستعرض مع وزير الدفاع بكوريا الجنو...,وكالة الأنباء السعودية (واس),الرياض,6,استقبل صاحب السمو الملكي الأمير محمد بن سلمان ...,"['المملكة', 'محليات']"
4,"رائدات أعمال لـ""سبق"": وضعنا بصمات مميزة عربي...",خلود غنام,الرياض,6,انطلق، مساء أمس، ملتقى سيدات الأعمال الـ19 الم...,"['المملكة', 'محليات']"


In [170]:
# Find the length of the news 
len(news)

4136

In [171]:
# Evaluate 
new = []
#import ast
for i in news.label:
    a = ast.literal_eval(i)
    #print(type(a))
    new.append(''.join(a))

In [172]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4136 entries, 0 to 4135
Data columns (total 6 columns):
title          4136 non-null object
author_name    4136 non-null object
city           4136 non-null object
shares         4136 non-null object
news_text      4134 non-null object
label          4136 non-null object
dtypes: object(6)
memory usage: 194.0+ KB


In [173]:
news['updated_label'] = new 

In [174]:
news[news['updated_label'] == 'كاريكاتير']

,title,author_name,city,shares,news_text,label,updated_label
3108,"""إرهاب نصر الله على المتظاهرين في لبنان""",صحيفة سبق الإلكترونية,الرياض,25,NaN,['كاريكاتير'],كاريكاتير
3893,توقيع ترامب يجبر أردوغان على الانسحاب من سوريا⁩,صحيفة سبق الإلكترونية,الرياض,40,NaN,['كاريكاتير'],كاريكاتير


In [175]:
# Drop the news with tilte "كاريكاتير" , as it is the reason for the missing in coulmn news_text
news.dropna(inplace=True)

In [176]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4134 entries, 0 to 4135
Data columns (total 7 columns):
title            4134 non-null object
author_name      4134 non-null object
city             4134 non-null object
shares           4134 non-null object
news_text        4134 non-null object
label            4134 non-null object
updated_label    4134 non-null object
dtypes: object(7)
memory usage: 258.4+ KB


In [177]:
#Check the Data after the cleaning 
news.head(3)

,title,author_name,city,shares,news_text,label,updated_label
0,الدفاع المدني بالرياض يناقش مع الهلال الأحمر...,صحيفة سبق الإلكترونية,الرياض,2,استقبل مدير الدفاع المدني بمنطقة الرياض، اللوا...,"['المملكة', 'محليات']",المملكةمحليات
1,"""أبوساق"": خطاب الملك بـ""الشورى"" وثيقة استرات...",وكالة الأنباء السعودية (واس),الرياض,4,عدَّ وزير الدولة عضو مجلس الوزراء لشؤون مجلس ا...,"['المملكة', 'محليات']",المملكةمحليات
2,المملكة ترفض تصريحات الحكومة الأمريكية بشأن ...,وكالة الأنباء السعودية (واس),الرياض,5,عبر مصدر مسؤول بوزارة الخارجية عن رفضه التام ل...,"['المملكة', 'محليات']",المملكةمحليات


# Step2 : Apply NLP 

In [178]:
pip install tashaphyne

Note: you may need to restart the kernel to use updated packages.


In [179]:
import pyarabic.arabrepr
arepr = pyarabic.arabrepr.ArabicRepr() 
repr = arepr.repr
from tashaphyne.stemming import ArabicLightStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [180]:
def review_to_words(raw_review):
    '''
    Function to convert a raw review to a string of words
    The input is a single string (a raw movie review), and 
    the output is a single string (a preprocessed movie review)
    '''
    ArListem = ArabicLightStemmer()
    
    # 1.Tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(raw_review)
    
    # 2. Stop Words list
    stopWords = open("list.txt").read().splitlines()
    stops = set(stopWords) #convert it to set for faster search 
            
    # 3. Remove stop words
    tokens = [i for i in tokens if not i in stops]
    
    # 4. Apply the ArListem.light_stem to the tokens list
    # which will almost return the root of the word
    
    stemmed_words = [ArListem.light_stem(i) for i in tokens]
    
    # and return the result.

    return(" ".join(stemmed_words))


## Applying the function "review_to_words" and creating a new list to append the data in it

In [181]:
print("Cleaning and parsing the training set news label......") # good to know what is going on!

# Initialize empty lists to hold the clean news.
clean_news = []

# Let's grab the number to run the loop!
total_news = news.shape[0]
print(f'There are {total_news} news_text.')

j = 0 #  counter
#for train_review in X_train['review']:

for text in news['news_text']:
    
    # Convert news to words, then append to clean_train_news.
    clean_news.append(review_to_words(text)) # <shift+tab> to see the doc for your own function!
    
    # If the index is divisible by 1000, print a message
    if (j + 1) % 100 == 0:
        print(f'Review {j + 1} of {total_news}.') 
    
    j += 1 # adding 1 to the counter 

print("All done!")

Cleaning and parsing the training set news label......
There are 4134 news_text.
Review 100 of 4134.
Review 200 of 4134.
Review 300 of 4134.
Review 400 of 4134.
Review 500 of 4134.
Review 600 of 4134.
Review 700 of 4134.
Review 800 of 4134.
Review 900 of 4134.
Review 1000 of 4134.
Review 1100 of 4134.
Review 1200 of 4134.
Review 1300 of 4134.
Review 1400 of 4134.
Review 1500 of 4134.
Review 1600 of 4134.
Review 1700 of 4134.
Review 1800 of 4134.
Review 1900 of 4134.
Review 2000 of 4134.
Review 2100 of 4134.
Review 2200 of 4134.
Review 2300 of 4134.
Review 2400 of 4134.
Review 2500 of 4134.
Review 2600 of 4134.
Review 2700 of 4134.
Review 2800 of 4134.
Review 2900 of 4134.
Review 3000 of 4134.
Review 3100 of 4134.
Review 3200 of 4134.
Review 3300 of 4134.
Review 3400 of 4134.
Review 3500 of 4134.
Review 3600 of 4134.
Review 3700 of 4134.
Review 3800 of 4134.
Review 3900 of 4134.
Review 4000 of 4134.
Review 4100 of 4134.
All done!


In [182]:
#Checking the item 0 in the clean_news list
clean_news[0]

'ستقبل مدير دفاع مدن منطق رياض لواء خالد حرق مدير إدار عام فرع هيئ هلال أحمر سعود منطق رياض سعود حرب ذلك مناقش سبل تعا إطار عمل ميدان جه دا لقاء كد لواء خالد حرق تعا جه مشدد هم عزيز تنسيق مدير دفاع مدن هلال أحمر منطق تخاذ تدابير إجراء لازم حما سلام مواطن مقيم مخاطر قديم مساعد ذليل مصاعب حصل رض ميد حرب شكر قدير واء خالد حرق حفاو استقبال كد تعا ظل مستمر إذن له عالى خدم مواطن مقيم ما خدم صالح عام لقاء سلم لواء حرق درع ذكار سعود حرب فيما حضر لقاء مساعد مدير دفاع مدن شؤو عمل منطق مدير إدار إطفاء إنقاذ عميد عبدالل سحيبان ضباط مدير هلال أحمر حضر مدير إدار قياد ميدان مشرف غرف عمل عبدالمحسن حرب مدير إدار اتصال مؤسس متحدث رسم هلال أحمر سعود منطق رياض ياسر جلاجل'

In [183]:
#Checking the length of the clean_news list 
len(clean_news)

4134

In [184]:
# Creating a coulmn named "cleaned" to include the 
news['cleaned']  = clean_news

In [185]:
#Checking The Data after updating the new column
news.head(5)

,title,author_name,city,shares,news_text,label,updated_label,cleaned
0,الدفاع المدني بالرياض يناقش مع الهلال الأحمر...,صحيفة سبق الإلكترونية,الرياض,2,استقبل مدير الدفاع المدني بمنطقة الرياض، اللوا...,"['المملكة', 'محليات']",المملكةمحليات,ستقبل مدير دفاع مدن منطق رياض لواء خالد حرق مد...
1,"""أبوساق"": خطاب الملك بـ""الشورى"" وثيقة استرات...",وكالة الأنباء السعودية (واس),الرياض,4,عدَّ وزير الدولة عضو مجلس الوزراء لشؤون مجلس ا...,"['المملكة', 'محليات']",المملكةمحليات,عد زير دول عض مجلس وزراء شؤو مجلس شورى محمد صل...
2,المملكة ترفض تصريحات الحكومة الأمريكية بشأن ...,وكالة الأنباء السعودية (واس),الرياض,5,عبر مصدر مسؤول بوزارة الخارجية عن رفضه التام ل...,"['المملكة', 'محليات']",المملكةمحليات,عبر مصدر مسؤول وزار خارج رفض تام تصريح حكوم أم...
3,ولي العهد يستعرض مع وزير الدفاع بكوريا الجنو...,وكالة الأنباء السعودية (واس),الرياض,6,استقبل صاحب السمو الملكي الأمير محمد بن سلمان ...,"['المملكة', 'محليات']",المملكةمحليات,ستقبل صاحب سمو ملك أمير محمد سلم عبدالعزيز لي ...
4,"رائدات أعمال لـ""سبق"": وضعنا بصمات مميزة عربي...",خلود غنام,الرياض,6,انطلق، مساء أمس، ملتقى سيدات الأعمال الـ19 الم...,"['المملكة', 'محليات']",المملكةمحليات,نطلق ملتقى سيد أعمال ـ19 مقام قاع خزامى مناسب ...


# Step3: Data is ready

**Vectorization using `CountVectorizer`:**<br>


In [186]:
from sklearn.feature_extraction.text import CountVectorizer

In [187]:
vectorizer = CountVectorizer(analyzer = "word", #<shift+tab> 
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000) 

In [188]:
data_features = vectorizer.fit_transform(clean_news)

In [189]:
vocab = vectorizer.get_feature_names()

In [190]:
data_features

<4134x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 428291 stored elements in Compressed Sparse Row format>

In [191]:
data_features = data_features.toarray()

In [192]:
df = pd.DataFrame(data_features, columns = vectorizer.get_feature_names())

In [193]:
df.head(2)

,00,000,03,04,06,07,10,100,1000,11,...,يوم,يونايتد,يونسكو,١١,١٤٤١هـ,٢٠١٩م,٢٠٣٠,٢٤,٣٠,٥٠
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Import train_test_split


In [194]:
# Import train_test_split.
from sklearn.model_selection import train_test_split

# Features and target
X = df
y = news['updated_label']

# Create train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42 )#, stratify = True)

# Applying LogisticRegression

In [195]:
from sklearn.linear_model import LogisticRegression

In [196]:
lr = LogisticRegression()

In [197]:
lr.fit(X_train, y_train)

/Users/asmaa/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/asmaa/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [198]:
lr.score(X_train, y_train)

1.0

In [199]:
lr.score(X_test, y_test)

0.879110251450677

 <font color='green'> __The lr score is almost good 0.88__   </font> 

## Try the Naive Bayes, using TF-IDF.

In [200]:
from sklearn.feature_extraction.text import TfidfTransformer

In [201]:
# Let's create an instance for TfidfTransformer and fit to the df -- See above, its your sparse df
tfidf_trans = TfidfTransformer()
tfidf_trans.fit(df)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [202]:
df_tfidf = tfidf_trans.transform(df)

# Importing the multinomial Naive Bayes model 

In [203]:
from sklearn.naive_bayes import MultinomialNB

In [204]:
# creating instance
model_nb = MultinomialNB() # <Shift+tab> for doc string

In [205]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df_tfidf, y, test_size = 0.25, random_state = 42)

In [206]:
# training on train data
model_nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [207]:
# Evaluate model on training data.
model_nb.score(X_train, y_train)

0.837741935483871

In [208]:
# Evaluate model on testing data.
model_nb.score(X_test, y_test)

0.839458413926499

 <font color='green'> __The nb score is almost good 0.84__   </font> 

In [209]:
# predictions for the test data
pred = model_nb.predict(X_test)

## Create the confusion matrix

In [210]:
from sklearn.metrics import classification_report, confusion_matrix
print (confusion_matrix(y_test, pred))
print (classification_report(y_test, pred))

[[ 81   0   0   0   0   0  26   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  15   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   4   0   0   0   7   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   7   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
 [ 11   0   0   0   0   0 729   0   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  11   1   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  11   0  48   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   3   0   2   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0

/Users/asmaa/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
